In [ ]:
!pip install bitsandbytes==0.43.1 trl peft==0.10.0 accelerate==0.28.0

In [ ]:
!pip install transformers==4.39.3

In [ ]:
!pip install flash-attn --no-build-isolation

  Using cached flash_attn-2.5.9.post1.tar.gz (2.6 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer







/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
system_prompt=""

In [ ]:
dataset = load_dataset("SamaaMaged-205/Tech_M_asamaa", split="train")
dataset

In [ ]:
df=dataset.to_pandas()

In [ ]:
df

In [ ]:
eval_df = df.sample(frac=0.1, random_state=42)

In [ ]:
remaining_df = df.drop(eval_df.index)

In [ ]:
train_dataset = Dataset.from_pandas(remaining_df)

In [ ]:
eval_dataset=Dataset.from_pandas(eval_df)

In [ ]:
train_dataset

In [ ]:
eval_dataset

In [ ]:
MODEL_NAME="unsloth/llama-3-8b-Instruct-bnb-4bit"

def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [ ]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

In [ ]:
for name, module in model.named_modules():
    print(name, module.__class__.__name__)

In [ ]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1

#["q_proj", "k_proj", "v_proj", "o_proj"]

peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=["q_proj", "k_proj"],
    bias="none",
    task_type="CAUSAL_LM",
    use_dora=True
)

In [ ]:

output_dir = './results'

# based on config
training_arguments = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=output_dir,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)


trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    dataset_text_field="merged_column",
    max_seq_length=64,
    tokenizer=tokenizer,
    args=training_arguments,
)

# To clear out cache for unsuccessful run
torch.cuda.empty_cache()



In [ ]:
train_result = trainer.train()